# EDA

In [67]:
import pandas as pd
import numpy as np

# Interactive plotting libraries
import plotly.express as px
import plotly.graph_objects as go

# For geospatial analysis
import plotly.io as pio
import plotly.offline as pyo

# To calculate distances
from geopy.distance import geodesic

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
data = pd.read_excel('fraud.xlsx')

# Display the first few rows
data.head()

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,2019-04-01 00:58:00,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-09-11 00:00:00,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
1,2019-04-01 15:06:00,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-09-11 00:00:00,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1
2,2019-04-01 22:37:00,Wisozk and Sons,misc_pos,49.61,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-09-11 00:00:00,86ba3a888b42cd3925881fa34177b4e0,65.347667,-165.914542,1
3,2019-04-01 23:06:00,Murray-Smitham,grocery_pos,295.26,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-09-11 00:00:00,3a068fe1d856f0ecedbed33e4b5f4496,64.445035,-166.080207,1
4,2019-04-01 23:59:00,Friesen Lt,health_fitness,18.17,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-09-11 00:00:00,891cdd1191028759dc20dc224347a0ff,65.447094,-165.446843,1


In [68]:
# Get information about data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14446 entries, 0 to 14445
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  14446 non-null  object 
 1   merchant               14446 non-null  object 
 2   category               14446 non-null  object 
 3   amt                    14446 non-null  float64
 4   city                   14446 non-null  object 
 5   state                  14446 non-null  object 
 6   lat                    14446 non-null  float64
 7   long                   14446 non-null  float64
 8   city_pop               14446 non-null  int64  
 9   job                    14446 non-null  object 
 10  dob                    14446 non-null  object 
 11  trans_num              14446 non-null  object 
 12  merch_lat              14446 non-null  float64
 13  merch_long             14446 non-null  float64
 14  is_fraud               14446 non-null  object 
dtypes:

In [69]:
# Check for missing values
data.isnull().sum()

,0
trans_date_trans_time,0
merchant,0
category,0
amt,0
city,0
state,0
lat,0
long,0
city_pop,0
job,0


In [70]:
data['is_fraud'].value_counts()

,count
is_fraud,
0,12600
1,1844
"1""2020-12-24 16:56:24""",1
"0""2019-01-01 00:00:44""",1


In [71]:
data['is_fraud'] = data['is_fraud'].replace({
    '1"2020-12-24 16:56:24"': 1,
    '0"2019-01-01 00:00:44"': 0
})

data['is_fraud'] = data['is_fraud'].astype(int)

print(data['is_fraud'].value_counts())

is_fraud
0    12601
1     1845
Name: count, dtype: int64


# Data Preprocessing

In [72]:
# Convert 'trans_date_trans_time' and 'dob' to datetime
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data['dob'] = pd.to_datetime(data['dob'], errors='coerce')

# Convert 'is_fraud' to integer
data['is_fraud'] = data['is_fraud'].astype(int)

In [73]:
# Calculate age
data['age'] = data['trans_date_trans_time'].dt.year - data['dob'].dt.year

In [74]:
# Extract hour from transaction time
data['hour'] = data['trans_date_trans_time'].dt.hour

In [75]:
# Extract day of week
data['day_of_week'] = data['trans_date_trans_time'].dt.day_name()

In [76]:
# Function to calculate distance
def calculate_distance(row):
    customer_location = (row['lat'], row['long'])
    merchant_location = (row['merch_lat'], row['merch_long'])
    return geodesic(customer_location, merchant_location).miles

# Apply the function
data['distance'] = data.apply(calculate_distance, axis=1)

In [77]:
# Sort data by customer and transaction time
data = data.sort_values(['trans_num', 'trans_date_trans_time'])

# Calculate time difference between transactions (assuming 'trans_num' is unique per transaction)
data['time_diff'] = data.groupby('trans_num')['trans_date_trans_time'].diff().dt.total_seconds()

# Calculate speed (distance/time_diff)
data['speed'] = data['distance'] / (data['time_diff'] / 3600)  # Convert seconds to hours

# Fill NaN values with 0 (first transaction will have NaN)
data['speed'].fillna(0, inplace=True)

In [78]:
data.head()

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,...,trans_num,merch_lat,merch_long,is_fraud,age,hour,day_of_week,distance,time_diff,speed
5941,2020-12-29 12:59:00,"""Witting, Beer and Ernse""",home,12.84,Cardwell,MO,36.0434,-90.2907,1120,Television floor manager,...,0003e99641f7a899fe6ec4ac9fb7a1c5,36.560368,-90.204879,0,63,12,Tuesday,35.965414,NaN,0.0
5185,2020-12-28 19:23:00,"""Crist, Jakubowski and Littel""",home,26.84,Kansas City,MO,38.9621,-94.5959,545147,Counsellor,...,001c13536e38d5d9c5268dfe02b7b3f3,38.154543,-93.954063,0,33,19,Monday,65.659019,NaN,0.0
7812,2019-01-01 02:12:00,"""Bernhard, Grant and Langworth""",shopping_pos,1.42,Westerville,NE,41.4193,-99.3844,73,Product designer,...,0020b02e5638ee80c7176007e1b483fa,40.790587,-99.011834,0,84,2,Tuesday,47.544316,NaN,0.0
4765,2020-12-28 11:47:00,Reichel Inc,entertainment,3.60,Laguna Hills,CA,33.6150,-117.7114,45303,Sales executive,...,0022405cbaac6b73479bc0e4b726089b,34.442366,-117.740463,0,51,11,Monday,57.050007,NaN,0.0
6928,2020-12-30 16:17:00,Turcotte-Halvorson,misc_pos,1.62,Smith River,CA,41.9404,-124.1587,1930,Web designer,...,0022a65b528b75e76836c77cae4ef5eb,42.213641,-123.796032,0,37,16,Wednesday,26.521634,NaN,0.0


In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14446 entries, 5941 to 266
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trans_date_trans_time  14446 non-null  datetime64[ns]
 1   merchant               14446 non-null  object        
 2   category               14446 non-null  object        
 3   amt                    14446 non-null  float64       
 4   city                   14446 non-null  object        
 5   state                  14446 non-null  object        
 6   lat                    14446 non-null  float64       
 7   long                   14446 non-null  float64       
 8   city_pop               14446 non-null  int64         
 9   job                    14446 non-null  object        
 10  dob                    14446 non-null  datetime64[ns]
 11  trans_num              14446 non-null  object        
 12  merch_lat              14446 non-null  float64       
 13  merch

# Univariate Analysis

Bagaimana distribusi jumlah transaksi?

In [103]:
nbins = 50

# Hitung batas bin menggunakan numpy
counts, bin_edges = np.histogram(data['amt'], bins=nbins)

# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    x='amt',
    nbins=nbins,
    title='Distribusi Jumlah Transaksi',  # Judul awal
    labels={'amt': 'Jumlah ($)'},
    text_auto=True
)

# Perbarui layout untuk mengatur judul menjadi tengah, tebal, dan besar
fig.update_layout(
    title={
        'text': "<b>Distribusi Jumlah Transaksi</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y':0.95,                                       # Posisi vertikal judul
        'x':0.5,                                        # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                                   # Ukuran font judul
            family="Arial, sans-serif",                # Jenis font
            color='black'                              # Warna font
        )
    },
    xaxis_title='Jumlah ($)',
    yaxis_title='Frekuensi',
    bargap=0.2  # Menambah jarak antar bar untuk visualisasi yang lebih baik
)

# Tampilkan plot
fig.show()

Insights:
1. Distribusi jumlah transaksi condong ke kanan (right-skewed).
2. Sejumlah kecil transaksi memiliki jumlah yang sangat tinggi.
3. Hal ini menunjukkan bahwa transaksi dengan nilai tinggi jarang terjadi tetapi mungkin signifikan dalam deteksi penipuan.


Bagaimana distribusi usia pelanggan?

In [118]:
# Tentukan jumlah bin
nbins = 50

# Hitung batas bin menggunakan numpy
counts, bin_edges = np.histogram(data['age'], bins=nbins)

# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    x='age',
    nbins=nbins,
    title='Distribution of Customer Age',  # Judul awal
    labels={'age': 'Age (years)'},
    text_auto=True  # Menampilkan frekuensi di atas setiap batang
)

# Perbarui layout untuk mengatur judul menjadi tebal, terpusat, dan berukuran besar
fig.update_layout(
    title={
        'text': "<b>Distribusi Umur Pelanggan</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y': 0.95,                                       # Posisi vertikal judul
        'x': 0.5,                                        # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                                   # Ukuran font judul
            family="Arial, sans-serif",                # Jenis font
            color='black'                              # Warna font
        )
    },
    xaxis_title='Age (years)',
    yaxis_title='Frequency',
    bargap=0.2  # Menambah jarak antar bar untuk visualisasi yang lebih baik
)

# Tampilkan plot
fig.show()

In [108]:
data['age'].mean()

48.12757856846186

Insights:
1. Sebagian besar pelanggan berusia antara 30 hingga 60 tahun.
2. Usia rata-rata pelanggan sekitar 48 tahun.
3. Memahami distribusi usia membantu mengidentifikasi kelompok usia yang mungkin lebih rentan terhadap penipuan.

Berapa banyak transaksi yang bersifat penipuan dibandingkan dengan yang tidak?

In [126]:
fig = px.histogram(
    data,
    x='is_fraud',
    color='is_fraud',
    title='<b>Jumlah Transaksi Fraud dan Non-Fraud</b>',  # Judul dalam Bahasa Indonesia
    text_auto=True,  # Menampilkan label jumlah di atas setiap batang
    labels={'is_fraud': 'Status Fraud'},  # Label sumbu X
    category_orders={'is_fraud': [0, 1]},  # Urutan kategori
    color_discrete_map={0: 'blue', 1: 'red'}  # Warna untuk masing-masing kategori
)

# Perbarui layout untuk mengatur judul, sumbu, dan gap antar batang
fig.update_layout(
    title={
        'text': "<b>Jumlah Transaksi Fraud vs Non-Fraud</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y': 0.95,                                       # Posisi vertikal judul
        'x': 0.5,                                        # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                                   # Ukuran font judul
            family="Arial, sans-serif",                # Jenis font
            color='black'                              # Warna font
        )
    },
    xaxis_title='Status Fraud',
    yaxis_title='Jumlah',
    xaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['Non-Fraud', 'Fraud']  # Mengganti label tick
    ),
    bargap=0.2,  # Menambahkan jarak antar batang (20% dari lebar batang)
    showlegend=True  # Menyembunyikan legend
)

# Tampilkan plot
fig.show()

Insights:
1. Dataset sangat tidak seimbang.
2. Sekitar 1845 transaksi adalah penipuan.
3. Ketidakseimbangan ini penting untuk dipertimbangkan dalam pemodelan dan analisis.

Apa kategori transaksi yang paling umum?

In [127]:
# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    y='category',
    color='category',
    title='Kategori Transaksi',  # Judul dalam Bahasa Indonesia
    text_auto=True,  # Menampilkan label jumlah di samping setiap batang
    labels={'category': 'Kategori'},  # Label sumbu Y
    category_orders={'category': data['category'].value_counts().index.tolist()},  # Urutan kategori berdasarkan jumlah
    color_discrete_sequence=px.colors.qualitative.Pastel  # Mengatur palet warna
)

# Perbarui layout untuk mengatur judul, sumbu, gap antar batang, dan format lainnya
fig.update_layout(
    title={
        'text': "<b>Kategori Transaksi</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y':0.95,                              # Posisi vertikal judul
        'x':0.5,                               # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                          # Ukuran font judul
            family="Arial, sans-serif",       # Jenis font
            color='black'                     # Warna font
        )
    },
    xaxis_title='Jumlah',
    yaxis_title='Kategori',
    bargap=0.2,  # Menambahkan jarak antar batang (20% dari lebar batang)
    showlegend=False  # Menyembunyikan legend karena warna sudah jelas merepresentasikan kategori
)

# Menyesuaikan tampilan label jumlah pada batang
fig.update_traces(
    orientation='h',  # Membuat batang horizontal
    textposition='auto',  # Menempatkan label secara otomatis
    marker_line_color='white',  # Menambahkan garis pemisah antar batang
    marker_line_width=1
)

# Tampilkan plot
fig.show()

Insights:
1. Kategori 'grocery_pos' dan 'shopping_pos' adalah yang paling sering muncul.
2. Memahami frekuensi kategori membantu mengidentifikasi di mana aktivitas penipuan terkonsentrasi.


Bagaimana distribusi jarak antara pelanggan dan merchant?

In [128]:
# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    x='distance',
    nbins=50,
    title='Distribusi Jarak antara Pelanggan dan Pedagang',  # Judul dalam Bahasa Indonesia
    text_auto=True,  # Menampilkan label jumlah di atas setiap batang
    labels={'distance': 'Jarak (miles)'},  # Label sumbu X
    color_discrete_sequence=['#636EFA']  # Mengatur warna batang (warna biru default)
)

# Perbarui layout untuk mengatur judul, sumbu, gap antar batang, dan format lainnya
fig.update_layout(
    title={
        'text': "<b>Distribusi Jarak antara Pelanggan dan Pedagang</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y':0.95,                              # Posisi vertikal judul
        'x':0.5,                               # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                          # Ukuran font judul
            family="Arial, sans-serif",       # Jenis font
            color='black'                     # Warna font
        )
    },
    xaxis_title='Jarak (miles)',
    yaxis_title='Frekuensi',
    bargap=0.2,  # Menambahkan jarak antar batang (20% dari lebar batang)
    plot_bgcolor='white',  # Mengatur latar belakang plot menjadi putih
    showlegend=False  # Menyembunyikan legend karena tidak diperlukan
)

# Menyesuaikan tampilan label jumlah pada batang
fig.update_traces(
    marker_color='#636EFA',  # Warna batang (sesuaikan sesuai kebutuhan)
    marker_line_color='white',  # Menambahkan garis pemisah antar batang
    marker_line_width=1,        # Ketebalan garis pemisah
    opacity=0.75                 # Menyesuaikan transparansi batang
)

# Tampilkan plot
fig.show()

Insights:
1. Jarak dihitung antara lokasi pelanggan dan merchant.
2. Sebagian besar transaksi terjadi dalam jarak pendek.
3. Ada transaksi dengan jarak yang besar, yang mungkin menunjukkan aktivitas tidak biasa.


Kapan transaksi biasanya terjadi selama hari?

In [133]:
mean_transactions = data['hour'].value_counts().mean()
print(f"Rata-rata transaksi per jam: {mean_transactions:.2f}")
# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    x='hour',
    nbins=24,
    title='Distribusi Transaksi berdasarkan Jam',  # Judul dalam Bahasa Indonesia
    text_auto=True,  # Menampilkan label jumlah di atas setiap batang
    labels={'hour': 'Jam dalam Sehari'},  # Label sumbu X
    color_discrete_sequence=['#FFA07A']  # Mengatur warna batang (warna salmon)
)

# Perbarui layout untuk mengatur judul, sumbu, gap antar batang, dan format lainnya
fig.update_layout(
    title={
        'text': "<b>Distribusi Transaksi berdasarkan Jam</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y':0.95,                              # Posisi vertikal judul
        'x':0.5,                               # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                          # Ukuran font judul
            family="Arial, sans-serif",       # Jenis font
            color='black'                     # Warna font
        )
    },
    xaxis_title='Jam dalam Sehari',
    yaxis_title='Frekuensi',
    bargap=0.2,  # Menambahkan jarak antar batang (20% dari lebar batang)
    plot_bgcolor='white',  # Mengatur latar belakang plot menjadi putih
    showlegend=False  # Menyembunyikan legend karena tidak diperlukan
)

# Menyesuaikan tampilan label jumlah pada batang
fig.update_traces(
    marker_color='#FFA07A',  # Warna batang (salmon)
    marker_line_color='white',  # Menambahkan garis pemisah putih antar batang
    marker_line_width=1,        # Ketebalan garis pemisah
    opacity=0.75                 # Menyesuaikan transparansi batang
)

# (Opsional) Menambahkan garis rata-rata transaksi per jam
fig.add_vline(
    x=np.mean(data['hour']),
    line=dict(color='green', width=2, dash='dash'),
    annotation_text='Rata-rata',
    annotation_position='top left',
    annotation_font_size=12,
    annotation_font_color='green'
)

# Tampilkan plot
fig.show()

Rata-rata transaksi per jam: 601.92


Insights:
1. Lebih banyak transaksi terjadi selama jam larut malam dan dini hari.
2. Transaksi lebih sering terjadi siang hari sampai dini hari

# Bivariate Analysis

Bagaimana jumlah transaksi berhubungan dengan kejadian penipuan?

In [136]:
import plotly.express as px

# Pastikan kolom 'is_fraud' hanya berisi 0 dan 1
assert data['is_fraud'].isin([0, 1]).all(), "Kolom 'is_fraud' harus hanya berisi 0 dan 1."

# Menghitung nilai median untuk anotasi
median_non_fraud = data[data['is_fraud'] == 0]['amt'].median()
median_fraud = data[data['is_fraud'] == 1]['amt'].median()

# Membuat box plot dengan Plotly Express
fig = px.box(
    data,
    x='is_fraud',
    y='amt',
    color='is_fraud',
    points='all',
    title='Jumlah Transaksi berdasarkan Kejadian Fraud',  # Judul dalam Bahasa Indonesia
    labels={'is_fraud': 'Kejadian Fraud', 'amt': 'Jumlah ($)'},
    category_orders={'is_fraud': [0, 1]},  # Urutan kategori
    color_discrete_map={0: 'steelblue', 1: 'crimson'},  # Warna untuk masing-masing kategori
    hover_data={'is_fraud': False}  # Menyembunyikan informasi 'is_fraud' pada hover
)

# Memperbarui layout untuk mengatur judul, sumbu, dan estetika lainnya
fig.update_layout(
    title={
        'text': "<b>Jumlah Transaksi berdasarkan Kejadian Fraud</b>",  # Membuat judul tebal dengan tag HTML <b>
        'y':0.95,                               # Posisi vertikal judul
        'x':0.5,                                # Posisi horizontal judul (0.5 = tengah)
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,                           # Ukuran font judul
            family="Arial, sans-serif",        # Jenis font
            color='black'                      # Warna font
        )
    },
    xaxis_title='Kejadian Fraud',
    yaxis_title='Jumlah ($)',
    xaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['Non-Fraud', 'Fraud'],
        tickfont=dict(
            size=14
        )
    ),
    yaxis=dict(
        tickfont=dict(
            size=14
        )
    ),
    plot_bgcolor='white',  # Mengatur latar belakang plot menjadi putih
    showlegend=False       # Menyembunyikan legend karena warna sudah merepresentasikan kategori
)

# Menyesuaikan tampilan box plot tanpa menggunakan marker_color yang berupa list
fig.update_traces(
    boxmean=True,                      # Menampilkan garis mean pada box plot
    marker_line_color='black',         # Menambahkan garis pemisah hitam antar box
    marker_line_width=1,               # Ketebalan garis pemisah
    opacity=0.7                         # Menyesuaikan transparansi box plot
)

# Menambahkan anotasi untuk median pada setiap kategori
fig.add_annotation(
    x=0,
    y=median_non_fraud,
    text=f"Median: {median_non_fraud:.2f}",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    font=dict(
        size=12,
        color='black'
    )
)

fig.add_annotation(
    x=1,
    y=median_fraud,
    text=f"Median: {median_fraud:.2f}",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    font=dict(
        size=12,
        color='black'
    )
)

# Tampilkan plot
fig.show()

Insights:
1. Transaksi penipuan cenderung memiliki jumlah yang lebih tinggi.
2. Jumlah median transaksi penipuan secara signifikan lebih tinggi daripada non-penipuan.
3. Ini menunjukkan bahwa transaksi dengan nilai tinggi lebih mungkin menjadi penipuan.

Apakah ada hubungan antara usia nasabah dan kejadian penipuan?

In [89]:
fig = px.box(
    data,
    x='is_fraud',
    y='age',
    color='is_fraud',
    points='all',
    title='Customer Age vs Fraud Occurrence'
)
fig.update_layout(
    xaxis_title='Is Fraud',
    yaxis_title='Age (years)',
    xaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['No Fraud', 'Fraud']
    ),
    showlegend=False
)
fig.show()

Insights:
1. Transaksi penipuan sedikit lebih umum terjadi di antara pelanggan yang lebih muda.Namun, perbedaannya tidak terlalu mencolok.
2. Usia saja mungkin bukan prediktor kuat penipuan.

Apakah jarak antara pelanggan dan merchant memengaruhi kejadian penipuan?

In [138]:
import plotly.express as px

# Buat box plot dengan Plotly Express
fig = px.box(
    data,
    x='is_fraud',
    y='distance',
    color='is_fraud',
    points='all',
    title='Distance vs Fraud Occurrence',
    labels={'is_fraud': 'Is Fraud', 'distance': 'Distance (miles)'}
)

# Perbarui layout untuk mengatur judul, sumbu, dan format lainnya
fig.update_layout(
    title={
        'text': "<b>Distance vs Fraud Occurrence</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,
            family="Arial, sans-serif",
            color='black'
        )
    },
    xaxis_title='Is Fraud',
    yaxis_title='Distance (miles)',
    xaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['No Fraud', 'Fraud']
    ),
    plot_bgcolor='white',
    showlegend=False
)

# Menyesuaikan warna box plot untuk kategori "No Fraud" dan "Fraud"
fig.update_traces(
    selector=dict(type='box', name='0'),  # Menyesuaikan warna "No Fraud"
    marker_color='blue',
    marker_line_color='white',
    marker_line_width=1,
    opacity=0.75
)
fig.update_traces(
    selector=dict(type='box', name='1'),  # Menyesuaikan warna "Fraud"
    marker_color='red',
    marker_line_color='white',
    marker_line_width=1,
    opacity=0.75
)

# Tampilkan plot
fig.show()

Insights:
1. Transaksi penipuan terjadi pada jarak tidak terlihat signifikan berbeda.

Apakah ada jam tertentu ketika penipuan lebih mungkin terjadi?

In [141]:
import plotly.express as px

# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    x='hour',
    color='is_fraud',
    barmode='group',
    nbins=24,
    title='Transaction Hour vs Fraud Occurrence',
    labels={'hour': 'Hour of Day', 'is_fraud': 'Is Fraud'},
    text_auto=True  # Menambahkan label jumlah pada setiap batang
)

# Perbarui layout untuk mengatur judul dan sumbu
fig.update_layout(
    title={
        'text': "<b>Transaction Hour vs Fraud Occurrence</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,
            family="Arial, sans-serif",
            color='black'
        )
    },
    xaxis_title='Jam per Hari',
    yaxis_title='Jumlah',
    plot_bgcolor='white'
)

# Menyesuaikan warna batang untuk kategori "No Fraud" (0) dan "Fraud" (1)
fig.update_traces(
    selector=dict(name='0'),  # Warna biru untuk "No Fraud"
    marker_color='blue',
    texttemplate='%{y}',      # Format label jumlah
    textposition='outside'    # Menampilkan label di luar batang
)
fig.update_traces(
    selector=dict(name='1'),  # Warna merah untuk "Fraud"
    marker_color='red',
    texttemplate='%{y}',      # Format label jumlah
    textposition='outside'    # Menampilkan label di luar batang
)

# Tampilkan plot
fig.show()

Insights:
1. Transaksi penipuan relatif lebih sering terjadi selama jam larut malam dan dini hari (12 pagi hingga 6 pagi).
2. Transaksi non-penipuan memuncak selama jam siang hari.
3. Waktu transaksi dapat menjadi fitur yang berguna dalam deteksi penipuan.

Apakah kategori transaksi tertentu lebih rentan terhadap penipuan?

In [142]:
import plotly.express as px

# Buat histogram dengan Plotly Express
fig = px.histogram(
    data,
    y='category',
    color='is_fraud',
    barmode='group',
    category_orders={'category': data['category'].value_counts().index.tolist()},
    title='Transaction Category vs Fraud Occurrence',
    labels={'category': 'Category', 'is_fraud': 'Is Fraud'},
    text_auto=True  # Menambahkan label jumlah pada setiap batang
)

# Perbarui layout untuk mengatur judul dan sumbu
fig.update_layout(
    title={
        'text': "<b>Transaction Category vs Fraud Occurrence</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,
            family="Arial, sans-serif",
            color='black'
        )
    },
    xaxis_title='Count',
    yaxis_title='Category',
    plot_bgcolor='white'
)

# Menyesuaikan warna batang untuk kategori "No Fraud" (0) dan "Fraud" (1)
fig.update_traces(
    selector=dict(name='0'),  # Warna biru untuk "No Fraud"
    marker_color='blue',
    texttemplate='%{x}',      # Format label jumlah
    textposition='outside'    # Menampilkan label di luar batang
)
fig.update_traces(
    selector=dict(name='1'),  # Warna merah untuk "Fraud"
    marker_color='red',
    texttemplate='%{x}',      # Format label jumlah
    textposition='outside'    # Menampilkan label di luar batang
)

# Tampilkan plot
fig.show()

Insights:
1. Kategori seperti 'grocery_pos' dan 'shopping_net' memiliki tingkat penipuan yang lebih tinggi.
2. Kategori dengan kehadiran fisik ('_pos') memiliki tingkat penipuan yang lebih rendah dibandingkan dengan kategori online ('_net').
3. Kategori transaksi adalah faktor penting dalam analisis penipuan.

# Multivariate Analysis

Bagaimana jumlah transaksi dan jarak bersama-sama berhubungan dengan kejadian penipuan?

In [143]:
import plotly.express as px

# Buat scatter plot dengan Plotly Express
fig = px.scatter(
    data,
    x='age',
    y='amt',
    color='is_fraud',
    title='Transaction Amount vs Customer Age Colored by Fraud',
    labels={'amt': 'Amount ($)', 'age': 'Age (years)', 'is_fraud': 'Is Fraud'},
    color_discrete_map={0: 'blue', 1: 'red'}  # Warna biru untuk "No Fraud" dan merah untuk "Fraud"
)

# Perbarui layout untuk bilah warna dan sumbu
fig.update_layout(
    title={
        'text': "<b>Transaction Amount vs Customer Age Colored by Fraud</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,
            family="Arial, sans-serif",
            color='black'
        )
    },
    xaxis_title='Age (years)',
    yaxis_title='Amount ($)',
    coloraxis_colorbar=dict(
        title='Is Fraud',
        tickvals=[0, 1],
        ticktext=['No Fraud', 'Fraud']
    )
)

# Tampilkan plot
fig.show()

Insights:
1. Transaksi dengan nilai tinggi pada umur yang besar lebih mungkin menjadi penipuan.
2. Transaksi non-penipuan berkumpul pada jumlah yang lebih rendah dan umur yang lebih pendek.
3. Interaksi ini menunjukkan bahwa menggabungkan jumlah dan umur meningkatkan deteksi penipuan.


Apakah ada pola dalam kejadian penipuan berdasarkan waktu dan hari dalam seminggu?

In [144]:
import plotly.express as px

# Menghitung jumlah fraud berdasarkan hari dan jam
fraud_counts = data.groupby(['day_of_week', 'hour'])['is_fraud'].sum().reset_index()
fraud_pivot = fraud_counts.pivot(index='day_of_week', columns='hour', values='is_fraud').fillna(0)

# Mengatur ulang urutan hari dalam minggu
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fraud_pivot = fraud_pivot.reindex(days_order)

# Membuat heatmap dengan Plotly Express
fig = px.imshow(
    fraud_pivot,
    aspect='auto',
    title='Heatmap of Fraud Occurrence by Hour and Day of Week',
    labels=dict(x='Hour of Day', y='Day of Week', color='Number of Frauds'),
    color_continuous_scale='Reds'  # Warna gradasi merah untuk menunjukkan frekuensi fraud
)

# Menampilkan label sumbu X di bagian atas heatmap
fig.update_xaxes(side="top")

# Menyesuaikan tata letak heatmap
fig.update_layout(
    title={
        'text': "<b>Heatmap of Fraud Occurrence by Hour and Day of Week</b>",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=20,
            family="Arial, sans-serif",
            color='black'
        )
    },
    xaxis_title='Hour of Day',
    yaxis_title='Day of Week',
    plot_bgcolor='white'
)

# Tampilkan heatmap
fig.show()

Insights:
1. Aktivitas penipuan memuncak selama larut malam dan dini hari, terutama pada akhir senin.
2. Sabtu dan Minggu menunjukkan jumlah penipuan yang menurun selama jam off-hours.
3. Pola ini menunjukkan bahwa pelaku penipuan mungkin memanfaatkan waktu ketika pemantauan kurang ketat.

Pekerjaan apa yang lebih terkait dengan penipuan?

In [145]:
import plotly.express as px

# Mendapatkan 10 pekerjaan terbanyak dan menyaring data sesuai pekerjaan tersebut
top_jobs = data['job'].value_counts().nlargest(10).index
job_data = data[data['job'].isin(top_jobs)]

# Membuat histogram dengan Plotly Express
fig = px.histogram(
    job_data,
    y='job',
    color='is_fraud',
    barmode='group',
    category_orders={'job': top_jobs},
    title='Top 10 Jobs vs Fraud Occurrence',
    labels={'job': 'Job Title', 'is_fraud': 'Is Fraud'},
    text_auto=True  # Menambahkan label jumlah pada setiap batang
)

# Perbarui layout untuk mengatur judul dan sumbu
fig.update_layout(
    title={
        'text': "<b>Top 10 Jobs vs Fraud Occurrence</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24,
            family="Arial, sans-serif",
            color='black'
        )
    },
    xaxis_title='Count',
    yaxis_title='Job Title',
    plot_bgcolor='white'
)

# Menyesuaikan warna batang untuk kategori "No Fraud" (0) dan "Fraud" (1)
fig.update_traces(
    selector=dict(name='0'),  # Warna biru untuk "No Fraud"
    marker_color='blue',
    texttemplate='%{x}',      # Format label jumlah
    textposition='outside'    # Menampilkan label di luar batang
)
fig.update_traces(
    selector=dict(name='1'),  # Warna merah untuk "Fraud"
    marker_color='red',
    texttemplate='%{x}',      # Format label jumlah
    textposition='outside'    # Menampilkan label di luar batang
)

# Tampilkan plot
fig.show()

Insights:
1. Profesi tertentu seperti "Surveyor" dan "Systems Analyst" memiliki jumlah transaksi penipuan yang lebih tinggi.
2. Hal ini dapat disebabkan oleh prevalensi pekerjaan ini dalam dataset atau mungkin mengindikasikan penargetan oleh penipu.

Apa korelasi antara variabel numerik dan penipuan?

In [96]:
numeric_cols = ['amt', 'age', 'distance', 'city_pop']
corr_matrix = data[numeric_cols + ['is_fraud']].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    title='Correlation Matrix',
    labels=dict(x='Variables', y='Variables', color='Correlation Coefficient')
)
fig.show()

Insights:
1. Terdapat korelasi positif antara 'amt' dan 'is_fraud' (~0,65).
2. 'age' juga menunjukkan korelasi positif dengan 'is_fraud' (~0,05).
3. Variabel-variabel lain memiliki korelasi yang lebih lemah, yang mengindikasikan bahwa variabel-variabel tersebut mungkin kurang dapat memprediksi dengan sendirinya.

Di mana transaksi penipuan terkonsentrasi secara geografis?

In [97]:
fraud_data = data[data['is_fraud'] == 1]

fig = px.scatter_mapbox(
    fraud_data,
    lat='lat',
    lon='long',
    hover_name='city',
    hover_data=['amt', 'category'],
    color_discrete_sequence=['red'],
    zoom=3,
    height=600,
    title='Locations of Fraudulent Transactions'
)
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r': 0, 't': 50, 'l': 0, 'b': 0})
fig.show()

Insights:
1. Transaksi penipuan tersebar di berbagai wilayah.
2. Beberapa wilayah menunjukkan konsentrasi yang lebih tinggi, mungkin karena populasi yang lebih tinggi atau penipuan yang ditargetkan.
3. Analisis geospasial dapat membantu memfokuskan upaya pencegahan penipuan di wilayah tertentu.